# 3. Quantify extracellular RNA

After defining extracellular transcripts (previous notebooks), we will now focus on characterizing and quantifying the presence of exRNA for the different genes profiled in the dataset employing different strategies and tests

## Import packages

In [2]:
import spatialdata as sd

import troutpy

/home/sergio/.local/lib/python3.10/site-packages/dask/dataframe/__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(
/home/sergio/.local/lib/python3.10/site-packages/numba/core/decorators.py:246: RuntimeWarning: nopython is set for njit and is ignored
  warnings.warn('nopython is set for njit and is ignored', RuntimeWarning)
/home/sergio/.local/lib/python3.10/site-packages/anndata/utils.py:429: FutureWarning: Importing read_text from `anndata` is deprecated. Import anndata.io.read_text instead.
  warnings.warn(msg, FutureWarning)


## Read the SpatialData object

We read the processed Spatialdata object, which was previously stored as .zarr

In [4]:
xenium_path_cropped = "/media/sergio/Discovair_final/mousebrain_prime_crop_communication_2.zarr"
sdata = sd.read_zarr(xenium_path_cropped)

## Testing for expression over noise levels

We implemented `troutpy.tl.quantify_overexpression` to identify **overexpressed genes relative to a noise threshold**.

Essentially, it computes a threshold based on the counts of specified control features and compares gene counts against this threshold to determine overexpression. The function calculates log-fold changes for each gene, annotates metadata with these results. It returns updated spatial data along with per-gene scores and the calculated threshold

In [5]:
control_codewords = ["negative_control_probe", "unassigned_codeword", "deprecated_codeword", "genomic_control_probe", "negative_control_codeword"]

troutpy.tl.quantify_overexpression(
    sdata,
    layer="transcripts",
    codeword_column="codeword_category",
    control_codewords=control_codewords,
    gene_id_column="feature_name",
    percentile_threshold=99.99,
)

Added 'xrna_metadata' table with 13035 unique genes to the SpatialData object.


## Testing for proportion of extracellular transcripts

The `troutpy.tl.extracellular_enrichment` function computes the proportions of extracellular and intracellular transcripts for each gene. 

Essentially, the function calculates: (1) the proportion of transcripts of each gene present extracellularly and (2) the log fold change of extracellular to intracellular proportions.  These results into the `sdata` object under the `xrna_metadata` layer.

In [6]:
troutpy.tl.extracellular_enrichment(sdata)

## Testing for Spatial Variability

The `troutpy.tl.spatial_variability` function quantifies the spatial variability of extracellular RNA using Moran's I, a metric for spatial autocorrelation.

In [7]:
troutpy.tl.spatial_variability(sdata, gene_id_key="feature_name", coords_keys=["x", "y"], n_neighbors=10, binsize=20)

Extracting gene counts: 100%|██████████| 13031/13031 [00:00<00:00, 16485.54it/s]


## Testing for local density

The `spatial_colocalization` function evaluates the spatial colocalization of extracellular RNA transcripts in spatial transcriptomics data by identifying regions where transcripts from the same or different genes are found in close proximity. The function calculates the proportion of colocalized transcripts for each gene, using a user-defined threshold to define colocalization, and integrates these results into the metadata of the dataset for downstream analysis.

The function calculates the proportion of colocalized transcripts for each gene based on a user-defined threshold, leveraging kernel density estimation to compute spatial gene expression densities. Results are integrated into the `sdata` object under the `xrna_metadata` layer.

In [8]:
troutpy.tl.spatial_colocalization(
    sdata, coords_keys=["x", "y"], gene_id_key="feature_name", resolution=1000, binsize=5, threshold_colocalized=1, copy=False
)

Extracting gene counts: 100%|██████████| 13031/13031 [00:07<00:00, 1814.27it/s]


## Save SpatialData as zarr

The resulting dataset is finally stored in a new zarr object

In [10]:
xenium_path_cropped = "/media/sergio/Discovair_final/mousebrain_prime_crop_communication.zarr"
sdata.write(xenium_path_cropped, overwrite=True)

INFO     The SpatialData object is not self-contained (i.e. it contains some elements that are Dask-backed from    
         locations outside /media/sergio/Discovair_final/mousebrain_prime_crop_communication.zarr). Please see the 
         documentation of `is_self_contained()` to understand the implications of working with SpatialData objects 
         that are not self-contained.                                                                              
INFO     The Zarr backing store has been changed from                                                              
         /media/sergio/Discovair_final/mousebrain_prime_crop_communication_2.zarr the new file path:               
         /media/sergio/Discovair_final/mousebrain_prime_crop_communication.zarr                                    


### ---TO DO--- Create visualization for the output of different tests

In [ ]:
xenium_path_cropped = "/media/sergio/Discovair_final/mousebrain_prime_crop_quantified.zarr"
sdata = sd.read_zarr(xenium_path_cropped)

In [ ]:
exrna_metrics = sdata["xrna_metadata"].var
# This ones are the most interesting ones, but others can be used
exrna_metrics_filt = exrna_metrics.loc[:, ["logfoldratio_over_noise", "logfoldratio_extracellular", "moran_I", "proportion_of_colocalized"]]

#### PLOTTING FUNCTION 1 (troutpy.pl)- lfr_over_noise grouped by gene type
- **Aim**: Represent the logfoldratio over noise for different genes
- **Input**: sdata["xrna_metadata"].var
- **Suggested Type of plot**: Either barplot, violin or stripplot, representing logfoldration over noise (`sdata["xrna_metadata"]['logfoldratio_over_noise']`),  grouping genes in based on wether they are control probes or not (in sdata["xrna_metadata"].var['control_probe'] column)
-  **Returns**: None (besides the plot) 

#### PLOTTING FUNCTION 2 (troutpy.pl) Sorted scatterplot/barplot 
- **Aim**: Represent, for a given metric, the genes sorted by them. (the logfoldratio extracellular or extracellular_proportion). Maybe highlighting top-bottom genes with highest-lower scores
- **Input**: sdata -- from which all info needed will be under `sdata["xrna_metadata"].var`, the name column in `sdata["xrna_metadata"].var` (str) (e.g. 'moran_i','extracellular_proportion')
- **Suggested Type of plot**: Either sorted scatter plot, stripplot,barplot or similar, representing extracellular proportion over genes. Maybe color based on  sdata["xrna_metadata"].var['control probe']
- **Return**: None

#### PLOTTING FUNCTION 3 (troutpy.pl)- Scatterplot
- **Aim**: For each gene, represent two of the metrics side by side, in the form of a scatter plot
- **Input**: sdata -- from which all info needed will be under sdata["xrna_metadata"].var, x and y axes
- **Suggested Type of plot**: Scatterplot, maybe colored by control probe column as well
(sdata["xrna_metadata"].var['control probe'])
- **Return**: None

#### PLOTTING FUNCTION 4 (troutpy.pl)- Summary visualization
- **Aim**: Represent in a tabular plot (heatmap, dotplot) the overall score for individual genes on different metrics.
- **Input**: sdata -- from which all info needed will be under `sdata["xrna_metadata"].var`, list of columns in `sdata["xrna_metadata"].var`, maybe list of genes to be visualized? if all are too many.
- **Suggested Type of plot**: Heatmap, dotplot or similar. Take sc.pl.dotplot/ sc.tl.matrixplot as reference?
- **Return**: None